In [3]:
import os
import calendar
import shutil
from PIL import Image
from PIL.ExifTags import TAGS
import cv2
from datetime import datetime

def get_taken_date(path):
    try:
        image = Image.open(path)
        exif_data = image._getexif()
        if exif_data:
            date_fields = ['DateTimeOriginal', 'DateTimeDigitized', 'DateTime']
            for tag, value in exif_data.items():
                decoded_tag = TAGS.get(tag, tag)
                if decoded_tag in date_fields:
                    try:
                        date_obj = datetime.strptime(value, "%Y:%m:%d %H:%M:%S")
                        print(f"[✓] Taken: {date_obj} - File: {os.path.basename(path)}")
                        return date_obj
                    except ValueError:
                        print(f"[!] Invalid date format: {value} - File: {os.path.basename(path)}")
        print(f"[!] No valid EXIF date: {os.path.basename(path)}")
        return None
    except Exception as e:
        print(f"[ERROR] Failed to read EXIF: {path} - Error: {e}")
        return None

def move_images_without_exif(folder_path):
    """Move images without EXIF data to a separate folder"""
    no_exif_folder = os.path.join(folder_path, "no_exif_images")
    
    # Create the no_exif_images folder if it doesn't exist
    if not os.path.exists(no_exif_folder):
        os.makedirs(no_exif_folder)
        print(f"[✓] Created folder for images without EXIF data: {no_exif_folder}")
    
    images_without_exif = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            path = os.path.join(folder_path, filename)
            date_taken = get_taken_date(path)
            if not date_taken:
                dest_path = os.path.join(no_exif_folder, filename)
                shutil.move(path, dest_path)
                images_without_exif.append(filename)
    
    if images_without_exif:
        print(f"\n[✓] Moved {len(images_without_exif)} images without EXIF data to {no_exif_folder}:")
        for img in images_without_exif:
            print(f"→ {img}")
    else:
        print("\n[✓] All images have EXIF data. No files moved.")
    
    return no_exif_folder

def get_images_sorted_by_date(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            path = os.path.join(folder_path, filename)
            date_taken = get_taken_date(path)
            if date_taken:
                images.append((path, date_taken))
    
    # Sort by month and day, regardless of year
    images.sort(key=lambda x: (x[1].month, x[1].day))

    print("\n[✔] Sorted image list:")
    for path, dt in images:
        print(f"→ {dt.strftime('%B %d')} - {os.path.basename(path)}")

    return images

def create_video_with_month_labels(images, output_path, fps=1, duration_per_image=5):
    if not images:
        print("[!] No images to process.")
        return

    frame = cv2.imread(images[0][0])
    height, width, _ = frame.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    month_names = list(calendar.month_name)[1:]  # ['January', ..., 'December']
    frames_per_image = int(fps * duration_per_image)

    for img_path, date_taken in images:
        frame = cv2.imread(img_path)
        resized_frame = cv2.resize(frame, (width, height))

        # Label with English month name
        month_text = month_names[date_taken.month - 1]
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 3
        color = (0, 255, 255)
        thickness = 5
        position = (50, 100)

        cv2.putText(resized_frame, month_text, position, font, font_scale, color, thickness, cv2.LINE_AA)

        for _ in range(frames_per_image):
            video.write(resized_frame)

    video.release()
    print(f"\n[✔] Video saved to: {output_path}")


def main():
    folder_path = r"C:\Users\weihua\Desktop\image2video\image2video\images"  # Change this to your image path
    output_path = 'output_video.mp4'

    # First, move images without EXIF data to a separate folder
    no_exif_folder = move_images_without_exif(folder_path)
    
    # Then process the remaining images with EXIF data
    images = get_images_sorted_by_date(folder_path)
    if images:
        create_video_with_month_labels(images, output_path, fps=1)
    else:
        print("[!] No images with EXIF data found. Video creation skipped.")

if __name__ == "__main__":
    main()

[✓] Created folder for images without EXIF data: C:\Users\weihua\Desktop\image2video\image2video\images\no_exif_images

[✓] All images have EXIF data. No files moved.

[✔] Sorted image list:
[!] No images with EXIF data found. Video creation skipped.
